In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [11]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Covid19.pdf')
docs=loader.load()
docs

Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 137 0 (offset 0)
Ignoring wrong pointing object 177 0 (offset 0)
Ignoring wrong pointing object 179 0 (offset 0)
Ignoring wrong pointing object 181 0 (offset 0)
Ignoring wrong pointing object 202 0 (offset 0)
Ignoring wrong pointing object 287 0 (offset 0)
Ignoring wrong pointing object 289 0 (offset 0)
Ignoring wrong pointing object 311 0 (offset 0)
Ignoring wrong pointing object 313 0 (offset 0)
Ignoring wrong pointing object 315 0 (offset 0)
Ignoring wrong pointing object 391 0 (offset 0)
Ignoring wrong pointing object 412 0 (offset 0)


[Document(metadata={'source': 'Covid19.pdf', 'page': 0}, page_content="The picture can't be \ndisplayed.\nThe picture can't \nbe displayed.\nWhat is COVID-19•COVID-19 is the infectious disease caused by the most recently discovered coronavirus1\n•This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019•COVID-19 is now a pandemic affecting many countries globally\n1https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub/q-a-detail/q-a-coronaviruses"),
 Document(metadata={'source': 'Covid19.pdf', 'page': 1}, page_content="The picture can't be \ndisplayed.\nThe picture can't \nbe displayed.\nImmunityPersonal measures \nDisease specificationsTesting \nTransmissionRisk groups Treatment Social & economicimpact\nContents\nThe \npictur\ne \ncan't \nbe \ndispla\nyed.\nThe \npict\nure \ncan'\nt be \ndisp\nlaye\nd.\nThe \npict\nure \ncan'\nt be \ndisp\nlaye\nd.\nThe \npict\nure \ncan'\nt be \ndisp\nlaye\nd.\nTh\ne \npic\n

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""],chunk_size=500,chunk_overlap=50)
final_docs=text_splitter.split_documents(docs)
final_docs

[Document(metadata={'source': 'Covid19.pdf', 'page': 0}, page_content="The picture can't be \ndisplayed.\nThe picture can't \nbe displayed.\nWhat is COVID-19•COVID-19 is the infectious disease caused by the most recently discovered coronavirus1\n•This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019•COVID-19 is now a pandemic affecting many countries globally\n1https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub/q-a-detail/q-a-coronaviruses"),
 Document(metadata={'source': 'Covid19.pdf', 'page': 1}, page_content="The picture can't be \ndisplayed.\nThe picture can't \nbe displayed.\nImmunityPersonal measures \nDisease specificationsTesting \nTransmissionRisk groups Treatment Social & economicimpact\nContents\nThe \npictur\ne \ncan't \nbe \ndispla\nyed.\nThe \npict\nure \ncan'\nt be \ndisp\nlaye\nd.\nThe \npict\nure \ncan'\nt be \ndisp\nlaye\nd.\nThe \npict\nure \ncan'\nt be \ndisp\nlaye\nd.\nTh\ne \npic\n

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Sandeep\Langchain-Krish_Naik\langchain\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Sandeep\Langchain-Krish_Naik\langchain\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sandeep\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate D

In [ ]:
## save to the disk
from langchain_chroma import Chroma
vectordb=Chroma.from_documents(final_docs,embedding=embedding_model,persist_directory="./chroma_db")


In [31]:
## load the disk
db2=Chroma(persist_directory="./chroma_db",embedding_function=embedding_model)
retriver=db2.as_retriever()


In [40]:
from langchain_huggingface import HuggingFaceEndpoint
llm=HuggingFaceEndpoint(
    repo_id="google/flan-t5-base",
    max_new_tokens=200,
    temperature=0.5,
    model_kwargs={"k":5},
    huggingfacehub_api_token=os.getenv('HF_TOKEN')
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [33]:
question="what is Covid 19?"


In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

results = rag_chain.invoke({"input":"what is covid19?" })

results

BadRequestError: (Request ID: nX__qJ5CesZhdvwZENE5t)

Bad request:
The following `model_kwargs` are not used by the model: ['return_full_text', 'stop', 'watermark', 'k', 'stop_sequences'] (note: typos in the generate arguments will also show up in this list)